In [1]:
import rope.base.project
from rope.refactor.extract import ExtractMethod
import re

In [6]:
APOSTROPHE_MARKER = "__APOSTROPHE__"
PERCENT_FORMAT_MARKER = "__PERCENT_FORMAT__"

def serialize(text: str):
    # Replace "'{var}'" with "__APOSTROPHE__{var}__APOSTROPHE__"
    text = re.sub(r"'{([^'}]*?)}'", f"{APOSTROPHE_MARKER}{{\\1}}{APOSTROPHE_MARKER}", text)
    # Replace "%s" with "__PERCENT_FORMAT__"
    text = re.sub(r"%\((.*?)\)s", f"{PERCENT_FORMAT_MARKER}{{\\1}}", text)
    return text

def deserialize(text: str):
    text = re.sub(f"{APOSTROPHE_MARKER}{{(.*?)}}{APOSTROPHE_MARKER}", "'{\\1}'", text)
    text = re.sub(f"{PERCENT_FORMAT_MARKER}{{(.*?)}}", "%(\\1)s", text)
    return text

myproject = rope.base.project.Project('../../')

myresource = myproject.get_resource('sweepai/utils/openai_proxy.py')
contents = myresource.read()
serialized_contents = serialize(myresource.read())
myresource.write(serialized_contents)
extract_span = r'''# we will try each region in order until we get a response
            # randomize the order of the list
            SHUFFLED_MULTI_REGION_CONFIG = random.sample(
                MULTI_REGION_CONFIG, len(MULTI_REGION_CONFIG)
            )
            for region_url, api_key in SHUFFLED_MULTI_REGION_CONFIG:
                try:
                    logger.info(
                        f"Calling {model} with engine {engine} on Azure url {region_url}."
                    )
                    openai.api_key = api_key
                    openai.api_base = region_url
                    openai.api_version = OPENAI_API_VERSION
                    openai.api_type = OPENAI_API_TYPE
                    response = openai.ChatCompletion.create(
                        engine=engine,
                        model=model,
                        messages=messages,
                        max_tokens=max_tokens,
                        temperature=temperature,
                        timeout=OPENAI_TIMEOUT,
                    )
                    return response["choices"][0].message.content
                except SystemExit:
                    raise SystemExit
                except Exception as e:
                    logger.exception(f"Error calling {region_url}: {e}")'''
serialized_extract_span = serialize(extract_span)

start, end = serialized_contents.find(serialized_extract_span), serialized_contents.find(serialized_extract_span) + len(serialized_extract_span)
print(start, end)

try:
    extractor = ExtractMethod(myproject, myresource, start, end)
    change_set = extractor.get_changes("helper")
    for change in change_set.changes:
        if change.old_contents is not None:
            change.old_contents = deserialize(change.old_contents)
        else:
            change.old_contents = deserialize(change.resource.read())
        change.new_contents = deserialize(change.new_contents)
    for change in change_set.changes:
        print(change.get_description())
except Exception as e:
    print(e)
finally:
    myresource.write(contents)

3408 4686
Return should be the last statement.


In [37]:
def extract_method(
    snippet,
    file_path,
    method_name,
    project_name="../../sweepai"
):
    project = rope.base.project.Project(project_name)

    resource = project.get_resource(file_path)
    contents = resource.read()
    start, end = contents.find(snippet), contents.find(snippet) + len(snippet)

    extractor = ExtractMethod(project, resource, start, end)
    change_set = extractor.get_changes(method_name, similar=True, global_=True)
    for change in change_set.changes:
        change.do()

    result = resource.read()

    for change in change_set.changes:
        change.undo()
    return result

snippet = r"""edit_sweep_comment(
                review_message + "\n\nSuccess! 🚀",
                4,
                pr_message=(
                    f"## Here's the PR! [{pr.html_url}]({pr.html_url}).\n{center(payment_message_start)}"
                ),
                done=True,
            )"""
print(extract_method(snippet, "handlers/on_ticket.py", "hydrate_sandbox"))

SyntaxError: f-string: invalid syntax (<string>, line 5)